# Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## 1 - CSV Processing Functions

### 1.1 Processing and Averaging Frontal or Back Electrodes

In [2]:
def process_and_average(arr, wave, placement="f"):
    """
    Get average power of either frontal or back electrodes for a given time
    
    Arguments:
        arr         -- CSV Data Frame for a trial of a person (Ex. Baseline recording for subject N)
        wave        --   String to identify wave type (Alpha, Beta, Theta, Gamma, Delta)
        placement   -- String to identify if it is back or front electrodes
    
    Returns:
        averaged_power -- (n_times) numpy array 
    """

    # Process Wave Identifier
    wave = wave.title()

    # Create temporary arrays to contain the AF7 and AF8 electrode recordings
    temp1 = None
    temp2 = None

    # Isolate the frontal electrodes
    if placement.lower() == "f":
        temp1 = arr[wave+"_AF7"]
        temp2 = arr[wave+"_AF8"]

    # Isolate the back electrodes
    elif placement.lower() == "b":
        temp1 = arr[wave+"_TP9"]
        temp2 = arr[wave+"_TP10"]

    # Invalid electrode placement
    else:
        raise Exception('Invalid Electrode Placement')
    
    # Drop all the NaN values
    temp1.dropna(axis = 0, inplace=True)
    temp2.dropna(axis = 0, inplace=True)

    # Average both electrode arrays
    avg = ( temp1 + temp2 ) / 2

    # Convert from dB to actual FFT values
    avg = 10**avg

    return avg.to_numpy()

### 1.2 Getting Specific Wave Type for all Trials

In [3]:
def get_wave(wave, placement, baseline, easy, medium, hard, extreme):
    """
    Return a specific wave from all trials
    
    Arguments:
        wave        -- String to identify wave type (Alpha, Beta, Theta, Gamma, Delta)
        placement   -- String to identify if it is back or front electrodes
        baseline    -- CSV Data Frame for all baseline trials
        easy        -- CSV Data Frame for all easy trials
        medium      -- CSV Data Frame for all medium trials
        hard        -- CSV Data Frame for all hard trials
        extreme     -- CSV Data Frame for all extreme trials
    Returns:
        data        -- (n_difficulty, n_subject, n_time) numpy array 
    """

    data    = []
    base_   = [process_and_average(i, wave, placement) for i in baseline]
    easy_   = [process_and_average(i, wave, placement) for i in easy]
    med_    = [process_and_average(i, wave, placement) for i in medium]
    hard_   = [process_and_average(i, wave, placement) for i in hard]
    ext_    = [process_and_average(i, wave, placement) for i in extreme]

    data.append(base_)
    data.append(easy_)
    data.append(med_)
    data.append(hard_)
    data.append(ext_)

    return data
    # Data[0] = Baseline data for all subjects with desired wave type
    # Data[0][0] = Baseline data for subject 1 with desired wave type

### 1.3 Processing all Data

In [4]:
baseline = []
easy = []
medium = []
hard = []
extreme = []

for i in range(1,3):
    baseline.append(pd.read_csv('Control/'+str(i)+'.csv'))
    easy.append(pd.read_csv('Easy/'+str(i)+'.csv'))
    medium.append(pd.read_csv('Medium/'+str(i)+'.csv'))
    hard.append(pd.read_csv('Hard/'+str(i)+'.csv'))
    extreme.append(pd.read_csv('Extreme/'+str(i)+'.csv'))

## 2 - Plotting Absolute Wave Types

### 2.1 Plotting Function

In [5]:
def plot_wave(wave, placement, baseline, easy, medium, hard, extreme):
    """
    Return a specific wave from all trials
    
    Arguments:
        wave        -- String to identify wave type (Alpha, Beta, Theta, Gamma, Delta)
        placement   -- String to identify if it is back or front electrodes
        baseline    -- CSV Data Frame for all baseline trials
        easy        -- CSV Data Frame for all easy trials
        medium      -- CSV Data Frame for all medium trials
        hard        -- CSV Data Frame for all hard trials
        extreme     -- CSV Data Frame for all extreme trials
    Returns:
        none        -- Prints Box Plots
    """

    # Gets Data for specific wave
    data = get_wave(wave, placement, baseline, easy, medium, hard, extreme)

    # Calculate mean
    mean_base = [i.mean() for i in data[0]]
    mean_easy = [i.mean() for i in data[1]]
    mean_med  = [i.mean() for i in data[2]]
    mean_hard = [i.mean() for i in data[3]]
    mean_ext  = [i.mean() for i in data[4]]

    # Calculate standard deviation
    std_base = [i.std() for i in data[0]]
    std_easy = [i.std() for i in data[1]]
    std_med  = [i.std() for i in data[2]]
    std_hard = [i.std() for i in data[3]]
    std_ext  = [i.std() for i in data[4]]

    # Plotting
    labels = [f'Subject {i+1}' for i in range(len(baseline))]
    x = np.arange(len(labels))
    width = 0.05

    fig, ax = plt.subplots(figsize=(12,6))
    rects1 = ax.bar(x,          mean_base, width, yerr=std_base, label = "Baseline", capsize = 5)
    rects2 = ax.bar(x+ width,   mean_easy, width, yerr=std_easy, label = "Easy", capsize = 5)
    rects3 = ax.bar(x+ 2*width, mean_med, width, yerr=std_med, label = "Medium", capsize = 5)
    rects4 = ax.bar(x+ 3*width, mean_hard, width, yerr=std_hard, label = "Hard", capsize = 5)
    rects5 = ax.bar(x+ 4*width, mean_ext, width, yerr=std_ext, label = "Extreme", capsize = 5)

    # Labels
    electrode_placement = ""
    if placement.lower() == "f":
        electrode_placement = "Front"
    else:
        electrode_placement = "Back"

    ax.set_ylabel(wave + " [dB]")
    ax.set_title(electrode_placement + " " + wave + " Levels")
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()
    plt.show()

### 2.2 Plotting Front Absolute Box Plots

In [ ]:
plot_wave("Alpha", "f", baseline, easy, medium, hard, extreme)

In [ ]:
plot_wave("Theta", "f", baseline, easy, medium, hard, extreme)

In [ ]:
plot_wave("Gamma", "f", baseline, easy, medium, hard, extreme)

In [ ]:
plot_wave("Beta", "f", baseline, easy, medium, hard, extreme)

### 2.3 Plotting Front Absolute Box Plots

In [ ]:
plot_wave("Alpha", "b", baseline, easy, medium, hard, extreme)

In [ ]:
plot_wave("Beta", "b", baseline, easy, medium, hard, extreme)

In [ ]:
plot_wave("Gamma", "b", baseline, easy, medium, hard, extreme)

In [ ]:
plot_wave("Theta", "b", baseline, easy, medium, hard, extreme)

## 3 - Plotting Relative Wave Types

### 3.1 Calculating Relative Waves

In [14]:
def calculate_relative(baseline, easy, medium, hard, extreme):
    """
    Return an array containing all relative powers
    
    Arguments:
        baseline    -- CSV Data Frame for all baseline trials
        easy        -- CSV Data Frame for all easy trials
        medium      -- CSV Data Frame for all medium trials
        hard        -- CSV Data Frame for all hard trials
        extreme     -- CSV Data Frame for all extreme trials
    Returns:
        relative_data -- (n_wave, n_difficulty, n_subjects, n_time) numpy array
    """

    # Gets all the Wave Types
    alpha_ = get_wave("Alpha", "f", baseline, easy, medium, hard, extreme)
    beta_ = get_wave("Beta", "f", baseline, easy, medium, hard, extreme)
    delta_ = get_wave("Delta", "f", baseline, easy, medium, hard, extreme)
    theta_ = get_wave("Theta", "f", baseline, easy, medium, hard, extreme)
    gamma_ = get_wave("Gamma", "f", baseline, easy, medium, hard, extreme)

    # 0-Alpha, 1-Beta, 2-Delta, 3-Theta, 4-Gamma
    relative_powers = [] #size - (power, difficulty, n_subject, n_time)
    
    # Alpha Relative
    alpha_rel = []
    for i in range(0, 5):
        power = []
        for j in range(len(baseline)):
            power.append(
                alpha_[i][j] / (alpha_[i][j] + beta_[i][j] + delta_[i][j] + theta_[i][j] + gamma_[i][j])
            )

        alpha_rel.append(power)

    # Beta Relative
    beta_rel = []
    for i in range(0, 5):
        power = []
        for j in range(len(baseline)):
            power.append(
                beta_[i][j] / (alpha_[i][j] + beta_[i][j] + delta_[i][j] + theta_[i][j] + gamma_[i][j])
            )

        beta_rel.append(power)

    # Delta Relative
    delta_rel = []
    for i in range(0, 5):
        power = []
        for j in range(len(baseline)):
            power.append(
                delta_[i][j] / (alpha_[i][j] + beta_[i][j] + delta_[i][j] + theta_[i][j] + gamma_[i][j])
            )

        delta_rel.append(power)

    # Theta Relative
    theta_rel = []
    for i in range(0, 5):
        power = []
        for j in range(len(baseline)):
            power.append(
                theta_[i][j] / (alpha_[i][j] + beta_[i][j] + delta_[i][j] + theta_[i][j] + gamma_[i][j])
            )

        theta_rel.append(power)

    # Gamma Relative
    gamma_rel = []
    for i in range(0, 5):
        power = []
        for j in range(len(baseline)):
            power.append(
                gamma_[i][j] / (alpha_[i][j] + beta_[i][j] + delta_[i][j] + theta_[i][j] + gamma_[i][j])
            )

        gamma_rel.append(power)


    relative_powers.append(alpha_rel)
    relative_powers.append(beta_rel)
    relative_powers.append(delta_rel)
    relative_powers.append(theta_rel)
    relative_powers.append(gamma_rel)

    return relative_powers

relative_data = calculate_relative(baseline, easy, medium, hard, extreme)

In [15]:
def plot_relative(wave, data):

    data_ = []
    if wave.lower() == "alpha":
        data_ = data[0]
    elif wave.lower() == "beta":
        data_ = data[1]
    elif wave.lower() == "delta":
        data_ = data[2]
    elif wave.lower() == "theta":
        data_ = data[3]
    elif wave.lower() == "gamma":
        data_ = data[4]
    else:
        raise Exception('Invalid Waveform')


    mean_base = [i.mean() for i in data_[0]]
    mean_easy = [i.mean() for i in data_[1]]
    mean_med  = [i.mean() for i in data_[2]]
    mean_hard = [i.mean() for i in data_[3]]
    mean_ext  = [i.mean() for i in data_[4]]

    std_base = [i.std() for i in data_[0]]
    std_easy = [i.std() for i in data_[1]]
    std_med  = [i.std() for i in data_[2]]
    std_hard = [i.std() for i in data_[3]]
    std_ext  = [i.std() for i in data_[4]]

    labels = [f'Subject {i+1}' for i in range(len(baseline))]
    x = np.arange(len(labels))
    width = 0.05

    fig, ax = plt.subplots(figsize=(12,6))
    rects1 = ax.bar(x,          mean_base, width, yerr=std_base, label = "Baseline", capsize = 5)
    rects2 = ax.bar(x+ width,   mean_easy, width, yerr=std_easy, label = "Easy", capsize = 5)
    rects3 = ax.bar(x+ 2*width, mean_med, width, yerr=std_med, label = "Medium", capsize = 5)
    rects4 = ax.bar(x+ 3*width, mean_hard, width, yerr=std_hard, label = "Hard", capsize = 5)
    rects5 = ax.bar(x+ 4*width, mean_ext, width, yerr=std_ext, label = "Extreme", capsize = 5)

    ax.set_ylabel(wave + " Relative Power [dB]")
    ax.set_title(wave + " Relative Levels")
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()
    plt.show()

In [ ]:
plot_relative("alpha", relative_data)

In [ ]:
plot_relative("beta", relative_data)

In [ ]:
plot_relative("delta", relative_data)

In [ ]:
plot_relative("theta", relative_data)

In [ ]:
plot_relative("gamma", relative_data)

## 4 - Plotting Engagement Level

In [ ]:
def plot_engagement(baseline, easy, medium, hard, extreme):
    # Beta / (Alpha + Theta)
    alpha_ = get_wave("Alpha", "f", baseline, easy, medium, hard, extreme)
    beta_ = get_wave("Beta", "f", baseline, easy, medium, hard, extreme)
    theta_ = get_wave("Theta", "f", baseline, easy, medium, hard, extreme)

    engagement = []
    
    for i in range(0, 5):
        engagement_ = []
        for j in range(len(baseline)):
            engagement_.append(beta_[i][j] / (alpha_[i][j] + theta_[i][j]))

        engagement.append(engagement_)

    mean_base = [i.mean() for i in engagement[0]]
    mean_easy = [i.mean() for i in engagement[1]]
    mean_med  = [i.mean() for i in engagement[2]]
    mean_hard = [i.mean() for i in engagement[3]]
    mean_ext  = [i.mean() for i in engagement[4]]

    std_base = [i.std() for i in engagement[0]]
    std_easy = [i.std() for i in engagement[1]]
    std_med  = [i.std() for i in engagement[2]]
    std_hard = [i.std() for i in engagement[3]]
    std_ext  = [i.std() for i in engagement[4]]

    labels = [f'Subject {i+1}' for i in range(len(baseline))]
    x = np.arange(len(labels))
    width = 0.05

    fig, ax = plt.subplots(figsize=(12,6))
    rects1 = ax.bar(x,          mean_base, width, yerr=std_base, label = "Baseline", capsize = 5)
    rects2 = ax.bar(x+ width,   mean_easy, width, yerr=std_easy, label = "Easy", capsize = 5)
    rects3 = ax.bar(x+ 2*width, mean_med, width, yerr=std_med, label = "Medium", capsize = 5)
    rects4 = ax.bar(x+ 3*width, mean_hard, width, yerr=std_hard, label = "Hard", capsize = 5)
    rects5 = ax.bar(x+ 4*width, mean_ext, width, yerr=std_ext, label = "Extreme", capsize = 5)

    ax.set_ylabel("Engagement [dB]")
    ax.set_title("Engagement Levels")
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()
    plt.show()
    
plot_engagement(baseline, easy, medium, hard, extreme)

## 5. Line Plot Visualization

### 5.1 Line Plot Visualization for A Specific Wave at a Specific Trial

In [55]:
def lineplot_wave(wave, placement, trial_type, trial_name):
    data_ = [process_and_average(i, wave, placement) for i in trial_type]
    
    min_length = min([len(i) for i in data_])
    trimmed_data = [i[:min_length] for i in data_]
    
    avg_data = np.mean(trimmed_data, axis=0)
    std_data = np.std(trimmed_data, axis=0)

    plt.figure(figsize=(14,8))
    for i in trimmed_data:
        plt.plot(i, alpha=0.6)

    plt.plot(avg_data, color = "r", label = "Average " + wave)
    
    plt.title(wave + " waves for " + trial_name + " trials")
    plt.xlabel("Time (seconds)")
    plt.ylabel("Power [dB]")
    plt.legend()
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
lineplot_wave("Alpha", "f", baseline, "Baseline")

In [ ]:
lineplot_wave("Beta", "f", medium, "Medium")

### 5.2 Line Plot Visualization of a subject

In [77]:
def lineplot_subject(subject, wave, placement, baseline, easy, medium, hard, extreme):
    data_ = get_wave(wave, placement, baseline, easy, medium, hard, extreme)
    if subject > len(data_[0]) or subject < 0:
        raise Exception('Invalid Subject')

    min_length = min([len(i[subject]) for i in data_])
    trimmed_data = [i[subject][:min_length] for i in data_]

    plt.figure(figsize=(14,8))
    plt.plot(trimmed_data[0], alpha=0.6, label="Baseline")
    plt.plot(trimmed_data[1], alpha=0.6, label="Easy")
    plt.plot(trimmed_data[2], alpha=0.6, label="Medium")
    plt.plot(trimmed_data[3], alpha=0.6, label="Hard")
    plt.plot(trimmed_data[4], alpha=0.6, label="Extreme")
    
    plt.title(wave + " waves for Subject " + f'{subject}')
    plt.xlabel("Time (seconds)")
    plt.ylabel("Power [dB]")
    plt.legend()
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
lineplot_subject(0, "Alpha", 'f', baseline, easy, medium, hard, extreme)

In [ ]:
lineplot_subject(1, "Beta", 'f', baseline, easy, medium, hard, extreme)

### 5.3 Line Plot Visualization for A Relative Waves at a Specific Trial

In [ ]:
def lineplot_engagement(subject, baseline, easy, medium, hard, extreme):
    # Beta / (Alpha + Theta)
    alpha_ = get_wave("Alpha", "f", baseline, easy, medium, hard, extreme)
    beta_ = get_wave("Beta", "f", baseline, easy, medium, hard, extreme)
    theta_ = get_wave("Theta", "f", baseline, easy, medium, hard, extreme)

    engagement = []
    
    for i in range(0, 5):
        engagement_ = []
        for j in range(len(baseline)):
            engagement_.append(beta_[i][j] / (alpha_[i][j] + theta_[i][j]))

        engagement.append(engagement_)

    min_length = min([len(i[subject]) for i in engagement])
    trimmed_data = [i[subject][:min_length] for i in engagement]

    plt.figure(figsize=(14,8))
    plt.plot(trimmed_data[0], alpha=0.6, label="Baseline")
    plt.plot(trimmed_data[1], alpha=0.6, label="Easy")
    plt.plot(trimmed_data[2], alpha=0.6, label="Medium")
    plt.plot(trimmed_data[3], alpha=0.6, label="Hard")
    plt.plot(trimmed_data[4], alpha=0.6, label="Extreme")

    plt.title("Engagement Index for Subject " + f'{subject}')
    plt.xlabel("Time (seconds)")
    plt.ylabel("Power [dB]")
    plt.legend()
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    plt.tight_layout()
    plt.show()

lineplot_engagement(1, baseline, easy, medium, hard, extreme)